In [54]:
import pandas as pd
import re
import numpy as np
import datetime

# 销售订单数据

In [55]:
# 预先处理好业务员、采购员名称、产品名称、地区

In [56]:
original_data = pd.read_excel('销售订单.xls', dtype=object)

In [57]:
f_data = original_data.copy()
# f_data = f_data[f_data['产品总价']!=0]
f_data = f_data[~((f_data['产品金额']==0)&(f_data['其他金额']==0))]
f_data['ID'] = list(range(len(f_data)))
f_data=f_data.reset_index(drop=True)
f_data

,合同号,合同总价,产品编码,产品数量,产品sku,产品中文名称,产品英文名称,产品分类,产品金额,其他金额,...,创建时间,订单状态,审核时间,采购员,是否采购,采购员地区,产品线,产品类型,备注,ID
0,ORP22237377,5540,123926,10000,123926-17,涤纶抽绳包,Budget Custom Drawstring Bag with Reinforced C...,包/抽绳包,5500,40,...,2022-02-17 13:38:35,已完成,2022-02-17 16:04:06,周盼盼,是,合肥,抽绳包,标准品,"返单，前期订单RP21060472(logo不一样）,走海运（有报价） 无明确交期 确认...",0
1,ORG22245176,620,123926,550,123926-18,涤纶抽绳包,Budget Custom Drawstring Bag with Reinforced C...,包/抽绳包,495,125,...,2022-02-18 13:58:51,待发货,2022-02-18 14:04:38,周盼盼,是,合肥,抽绳包,标准品,2色印刷收65美金运费 (走快递）（有报价） 有明确交期3.9,1
2,ORG22336866,149,123926,100,123926-16,涤纶抽绳包,Budget Custom Drawstring Bag with Reinforced C...,包/抽绳包,84,65,...,2022-03-04 14:55:16,已完成,2022-03-04 15:43:13,周盼盼,是,合肥,抽绳包,标准品,收25美金运费 走快递（有报价） 有明确交期3.18， 确认并付款后生产5个工作日，但由于...,2
3,ORP22274942,380,123975,250,123975-17,热封口抽绳袋,Large Non-Woven Drawstring Backpack,包/抽绳包,285,95,...,2022-02-15 13:18:36,已完成,2022-02-15 17:05:06,周盼盼,是,合肥,抽绳包,标准品,"返单，前期订单 RP21020562,收60美金运费（快递）（有报价） 有明确交期3.2,...",3
4,ORP22213390,190.8,122168,250,122168-15,龙虾扣+插扣挂绳,"3/4"" Full Color Dye Sublimated Lanyard w/ Lobs...",活动与节日庆祝/挂绳 & 证件套/挂绳,140,50.8,...,2022-02-18 16:27:45,已收货,2022-02-18 16:28:37,马威,是,苏州,挂绳,标准品,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8986,ORP22693889,23275,301728,175000,301728-01,磁贴,MagnetCard,活动与节日庆祝/赛事 & 比赛用品,23275,0,...,2022-06-22 11:51:12,已通过,2022-06-24 09:01:39,NaN,否,NaN,NaN,订制品,NaN,8986
8987,ORP22657763,640,301729,500,301729-01,手机支架,Cell Phone Holder,电子与科技/手机配件/手机支架,600,40,...,2022-06-24 07:15:19,已通过,2022-06-24 09:51:48,NaN,否,NaN,NaN,订制品,NaN,8987
8988,ORP22652856,780,301730,50,301730-01,旅行包三件套,Packing Cubes 3 In 1,包/旅行包 & 附件/服装袋,730,50,...,2022-06-24 09:46:32,待发货,2022-06-24 10:40:46,陈坤,是,苏州,NaN,订制品,NaN,8988
8989,ORP22621984,369,301731,110,301731-01,棒球帽,CUSTOM HAT,服装&配饰/帽子/棒球帽,319,50,...,2022-06-24 10:20:25,待发货,2022-06-24 11:15:04,徐凯,是,苏州,NaN,订制品,NaN,8989


In [58]:
# print((len(f_data),len(original_data[original_data['产品总价']==0])))
print((len(original_data[~((original_data['产品金额']==0)&(original_data['其他金额']==0))]), len(original_data[(original_data['产品金额']==0)&(original_data['其他金额']==0)])))

(8991, 145)


In [59]:
print(len(original_data[(original_data['产品金额']==0)&(original_data['其他金额']==0)]))

145


## 产品分类

In [60]:
def divide_type(x):
    if type(x) == str and len(x) > 0:
        regex_three = '(.*?)/(.*?)/(.*)'
        regex_two = '(.*?)/(.*)'   # 非贪婪模式，/后的内容会取不到
        result_three = re.compile(regex_three, re.S).search(x)
        result_two = re.compile(regex_two, re.S).search(x)
        if result_three:
            class_one = result_three.group(1)
            class_two = result_three.group(2)
            class_three = result_three.group(3)
            return class_one,class_two,class_three
        if result_two:
            class_one = result_two.group(1)
            class_two = result_two.group(2)
            class_three = result_two.group(2)
            return class_one,class_two,class_three
        if '/' not in x:
            class_one = x
            class_two = x
            class_three = x
            return class_one,class_two,class_three
    return x

In [61]:
f_data['一级分类'] = f_data['产品分类']
f_data['二级分类'] = f_data['产品分类'].apply(lambda x: divide_type(x)[1])

## 产品总价

In [62]:
f_data['产品总价'] = f_data['产品金额']+f_data['其他金额']

## 产品颜色

In [63]:
f_data['产品颜色'] = f_data['产品规格']

## 采购

In [64]:
manager_list = pd.read_excel('E:/code/pbi_data/业务员.xlsx', dtype=object)

In [65]:
f_data['业务员'] = f_data['业务员英文名']

In [66]:
f_data = f_data.drop(['是否采购', '采购员地区'], axis=1)
f_data['采购员'] = f_data['采购员'].fillna('未分配')
f_data[f_data['采购员'].isnull()]

,合同号,合同总价,产品编码,产品数量,产品sku,产品中文名称,产品英文名称,产品分类,产品金额,其他金额,...,审核时间,采购员,产品线,产品类型,备注,ID,一级分类,二级分类,产品总价,产品颜色


In [67]:
f_data.rename(columns={'采购员':'采购员中文'},inplace=True)
maneger_data = pd.merge(f_data[['ID', '采购员中文', ]], manager_list,left_on = '采购员中文', right_on = '业务员', how='left')
maneger_data

,ID,采购员中文,业务员,地区,英文名,采购,是否采购,名,英中名
0,0,周盼盼,周盼盼,合肥,Phoebe Zhou,Phoebe Zhou,是,Phoebe,Phoebe 周盼盼
1,1,周盼盼,周盼盼,合肥,Phoebe Zhou,Phoebe Zhou,是,Phoebe,Phoebe 周盼盼
2,2,周盼盼,周盼盼,合肥,Phoebe Zhou,Phoebe Zhou,是,Phoebe,Phoebe 周盼盼
3,3,周盼盼,周盼盼,合肥,Phoebe Zhou,Phoebe Zhou,是,Phoebe,Phoebe 周盼盼
4,4,马威,马威,苏州,Jesse Ma,Jesse Ma,是,Jesse,Jesse 马威
...,...,...,...,...,...,...,...,...,...
8986,8986,未分配,未分配,NaN,未分配,其他,否,未分配,未分配
8987,8987,未分配,未分配,NaN,未分配,其他,否,未分配,未分配
8988,8988,陈坤,陈坤,苏州,Kun Chen,Kun Chen,是,Kun,Kun 陈坤
8989,8989,徐凯,徐凯,苏州,Kevin Xu,Kevin Xu,是,Kevin,Kevin 徐凯


In [68]:
f_data.loc[maneger_data['ID'], '采购员'] = maneger_data['英文名']
f_data.loc[maneger_data['ID'], '采购'] = maneger_data['采购']
f_data.loc[maneger_data['ID'], '是否采购'] = maneger_data['是否采购']
#  是采购的，匹配对应地区
f_data.loc[maneger_data['ID'], '采购员地区'] = maneger_data[maneger_data['是否采购']=='是']['地区']
f_data.loc[f_data['采购员']=='未分配', '是否已分配']='未分配'
f_data.loc[f_data['采购员']!='未分配', '是否已分配']='已分配'

In [69]:
f_data[f_data['采购员'].isnull()]['采购员中文']

Series([], Name: 采购员中文, dtype: object)

In [70]:
# f_data[f_data['采购员'].isnull()].to_excel('check.xlsx', index=False)

## 产线

In [71]:
product_line_data = pd.read_excel('E:/code/pbi_data/产线归属.xlsx', dtype=object)

len(f_data[f_data['产品线'].notnull()])

7327

In [72]:
# 原始数据产品线为空的，赋值为非产线
f_data['产品线'] = f_data['产品线'].fillna('非产线')

In [73]:
# 产品线2不为空且不为非产线，即在公司产品线的数据
# f_data.rename(columns={'产品线': '产品线2'}, inplace=True)
f_productline = f_data[(f_data['产品线'].notnull()) & (f_data['产品线'] != '非产线')]
productline_match_data = pd.merge(f_productline[['ID', '地区', '产品分类', '产品线']],
                                  product_line_data,
                                  left_on='产品线',
                                  right_on='产品线2_0',
                                  how='left')

In [74]:
len(f_productline)

7326

In [75]:
# 产品线2为空或为非产线，即不在公司产品线的数据
f_productline_no = f_data[(f_data['产品线'].isnull())|(f_data['产品线']=='非产线')]

In [76]:
# 产品名称需核查：产品线2不为空但产线归属为空
productline_match_data[productline_match_data['产品线2'].notnull() & productline_match_data['产品线归属'].isnull()]

productline_match_data.index = productline_match_data['ID']

In [77]:
# 地区与产线归属匹配的，直接赋值产品线2的值
temp1 = productline_match_data[productline_match_data['地区'] == productline_match_data['产品线归属']].copy()
temp1['产品线']= temp1['产品线2']
# 地区与产线归属不匹配的，赋值为非产线
temp2 = productline_match_data[productline_match_data['地区'] != productline_match_data['产品线归属']].copy()
temp2['产品线'] = '非产线'
productline_match_data.loc[temp1['ID'], '产品线']= temp1['产品线']
productline_match_data.loc[temp2['ID'], '产品线']= temp2['产品线']

# 是否产线。该数据为公司产线数据，故直接赋值为产线
productline_match_data['产品线归属'] = '产线'

#产品类型
productline_match_data.loc[:, '产品类型'] = productline_match_data['产品线2']

In [78]:
f_data.loc[productline_match_data['ID'], '产品线'] = productline_match_data['产品线']
f_data.loc[productline_match_data['ID'], '产品类型'] = productline_match_data['产品类型']
f_data.loc[productline_match_data['ID'], '产品线2'] = productline_match_data['产品线2']
f_data.loc[productline_match_data['ID'], '是否产品线'] = productline_match_data['产品线归属']

In [79]:
f_data['辅助项']= ''

In [80]:
f_productline_no['产品线'] = '非产线'

f_productline_no.loc[:,'产品类型'] =f_productline_no['产品分类'].apply(lambda x: divide_type(x)[2])
f_productline_no['产品线归属'] = '非产线'

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12328/4280139119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_productline_no['产品线'] = '非产线'
D:\anaconda\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_12328/4280139119.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [81]:
f_data['产品名称'] = f_data['产品中文名称']

In [82]:
f_data.loc[f_productline_no['ID'], '产品线'] = f_productline_no['产品线']
f_data.loc[f_productline_no['ID'], '产品类型'] = f_productline_no['产品类型']
f_data.loc[f_productline_no['ID'], '产品线2'] = f_productline_no['产品线']
f_data.loc[f_productline_no['ID'], '是否产品线'] = f_productline_no['产品线归属']

f_data[f_data['产品类型'].isnull()]

,合同号,合同总价,产品编码,产品数量,产品sku,产品中文名称,产品英文名称,产品分类,产品金额,其他金额,...,产品颜色,采购员,采购,是否采购,采购员地区,是否已分配,产品线2,是否产品线,辅助项,产品名称


## 表头

In [83]:
header = pd.read_excel('./辅助数据/表头.xlsx',dtype=object)

In [84]:
final_data = f_data[list(header)]

In [86]:
print((len(original_data[~((original_data['产品金额']==0)&(original_data['其他金额']==0))]), len(original_data[(original_data['产品金额']==0)&(original_data['其他金额']==0)])))

(8991, 145)


In [85]:
len(final_data)

8991

In [87]:
final_data.to_excel('维护结果.xlsx', index=False)

# 内销数据

In [88]:
# 一级分类、二级分类、产品线2、产品类型自行创建

In [89]:
original_nxdata = pd.read_excel('阿里巴巴.xlsx', dtype=object)
original_nxdata.head(3)

,平台渠道,店铺账号,下单时间,付款时间,订单号,SKU,产品ID,产品名称,一级分类,二级分类,...,订单备注,退款金额,币种缩写,付款方式,英文报关名,申报单价,中文报关名,商品采购价,产品线2,产品类型
0,手工订单,手工订单,2022-06-17 16:05:33,2022-06-17 16:05:33,2703244863950733754,1688EWxb211122-24,946505,战术包胸挂包,包,胸包,...,NaN,NaN,CNY,NaN,11,1.0,1,240.0,胸包,胸包
1,手工订单,手工订单,2022-06-17 16:20:57,2022-06-17 16:20:57,2700534278089476572,A204467-WHITE,946505,冰袖白,服装和配件,配件,...,NaN,NaN,CNY,NaN,ice sleeve,10.0,冰袖,2.55,非产线,其它/小类
2,手工订单,手工订单,2022-06-20 10:09:56,2022-06-20 10:09:56,220618-320646706192296,pddEWxb211116-40,946505,跨境野营军迷彩战术腿包腿挂 户外登山钓鱼骑行腿包 摄影运动腿包,包,运动包,...,NaN,NaN,CNY,NaN,bag,1.0,腿包,0.0,非产线,非产线


In [90]:
f_nxdata = original_nxdata.copy()
f_nxdata['ID'] = list(range(len(f_nxdata)))

## 字段填充

In [91]:
# 字段填充
# 合同号
f_nxdata['合同号'] = f_nxdata['订单号'].apply(lambda x: "NX" + str(x))

# 产品编码
f_nxdata['产品编码'] = f_nxdata['商品SKU']

#创建时间
f_nxdata['创建时间'] = f_nxdata['下单时间']

# 地区
f_nxdata['地区'] = '苏州'

# 产品总价
f_nxdata['产品售价'] = f_nxdata['产品售价'].astype(float)
f_nxdata['产品总价'] = f_nxdata['产品售价']*f_nxdata['产品数量']/6.5
f_nxdata['币种'] = '美金(换算)'
f_nxdata['税率'] = 6.5

f_nxdata['产品颜色'] = np.nan
f_nxdata['业务员'] = '阿里巴巴'
f_nxdata['客户编码'] = np.nan
f_nxdata['客户名称'] = np.nan
f_nxdata['客户邮箱'] = np.nan
f_nxdata['公司名称'] = '阿里巴巴'
f_nxdata['联系电话'] = np.nan
f_nxdata['辅助项'] = np.nan
f_nxdata['采购员'] = '已分配'
f_nxdata['采购'] = '其他'
f_nxdata['是否采购'] = '否'
f_nxdata['是否已分配'] = '已分配'
f_nxdata['采购员地区'] = np.nan
f_nxdata['备注'] = f_nxdata['订单备注']

## 产线

In [92]:
productline_match_nxdata = pd.merge(f_nxdata[['ID', '地区', '产品名称', '产品线2']],
                                    product_line_data[['产品线2', '产品线归属']],
                                    on='产品线2',
                                    how='left')

productline_match_nxdata.index = productline_match_nxdata['ID']

productline_match_nxdata = productline_match_nxdata.drop_duplicates()
productline_match_nxdata

,ID,地区,产品名称,产品线2,产品线归属
ID,,,,,
0,0,苏州,战术包胸挂包,胸包,合肥
1,1,苏州,冰袖白,非产线,NaN
2,2,苏州,跨境野营军迷彩战术腿包腿挂 户外登山钓鱼骑行腿包 摄影运动腿包,非产线,NaN
3,3,苏州,健身防水运动跑步腰包户外便捷斜跨水壶腰包登山贴身包,非产线,NaN
4,4,苏州,彩虹条纹镭射双肩包-紫色彩条,双肩包,合肥
5,5,苏州,运动健身男女户外腰包时尚斜挎包多色骑行徒步专业潮流通用,腰包/臂包,苏州
6,6,苏州,折叠双肩包,折叠包,苏州
7,7,苏州,折叠双肩包,折叠包,苏州
8,8,苏州,彩虹条纹镭射双肩包-紫色彩条,双肩包,合肥


In [93]:
# 产品线
tempnx1 = productline_match_nxdata[productline_match_nxdata['地区'] == productline_match_nxdata['产品线归属']].copy()
tempnx1['产品线']= tempnx1['产品线2']
tempnx2 = productline_match_nxdata[productline_match_nxdata['地区'] != productline_match_nxdata['产品线归属']].copy()
tempnx2['产品线'] = '非产线'
productline_match_nxdata.loc[tempnx1['ID'], '产品线']= tempnx1['产品线']
productline_match_nxdata.loc[tempnx2['ID'], '产品线']= tempnx2['产品线']

In [94]:
# 是否产品线
productline_match_nxdata['是否产品线'] = productline_match_nxdata['产品线归属'].apply(lambda x: '非产线' if len(str(x))!=2 else '产线')

In [95]:
f_nxdata.loc[productline_match_nxdata['ID'], '产品线'] = productline_match_nxdata['产品线']
f_nxdata.loc[productline_match_nxdata['ID'], '是否产品线'] = productline_match_nxdata['是否产品线']

In [96]:
f_nxdata['创建时间'] = f_nxdata['创建时间'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [97]:
f_nxdata['产品英文名称'] = ''

In [98]:
final_nxdata = f_nxdata[list(header)]

In [99]:
final_nxdata.to_excel('内销维护结果.xlsx', index=False)

# Tom数据

In [100]:
tom_header = pd.read_excel('./辅助数据/Tom数据表头.xlsx')

In [101]:
start_date = datetime.datetime.strptime('2022/6/1', '%Y/%m/%d')
end_date = datetime.datetime.strptime('2022/6/30', '%Y/%m/%d')

In [102]:
tom_data = final_data[list(tom_header)]

In [103]:
tom_data = tom_data[(tom_data['创建时间']>=start_date)&(tom_data['创建时间']<=end_date)]

In [104]:
tom_data.to_excel('Tom合同数据.xlsx', index=False)

In [105]:
len(tom_data)

1932